In [71]:
import json

In [72]:
fname = 'input-files//tweets.json'
with open(fname) as f:
    allTweets = [x.strip() for x in f] 

In [73]:
tweets = []

for tweet in allTweets:
    tweetDict = json.loads(tweet)
    tweets.append(tweetDict)
    
# print(tweets)

In [74]:
tweetIDs = []
tweetText = []

for tweet in tweets:
    tweetIDs.append(tweet['id_str'])
    tweetText.append(tweet['text'])

In [75]:
def getJaccardDistance(a, b):
#     if(a == b):
#         return 0
    
    a = a.split()
    b = b.split()
#     print("String 1", a)
#     print("String 2", b)
    
    common_words = list(set([word for word in a if word in b]))
    all_words = list(set(a).union(b))
#     print(common_words)
#     print(all_words)
    
    intersect = len(common_words)
    union = len(all_words)
#     print(intersect)
#     print(union)
    return (1 - intersect/union)

In [76]:
a = "the long may"
b = "ides of march"
getJaccardDistance(a, b)

1.0

In [77]:
getJaccardDistance(tweetText[0], tweetText[0])

0.0

In [78]:
a = tweetText[0]
b = tweetText[1]

In [79]:
getJaccardDistance(a, b)

0.17391304347826086

In [80]:
totalTweets = len(tweetText)

In [81]:
arrJD = [[] for i in range(totalTweets)]

In [82]:
for i in range(len(arrJD)):
    for j in range(len(arrJD)):
        arrJD[i].append(getJaccardDistance(tweetText[i], tweetText[j]))

In [83]:
arrJD

[[0.0,
  0.17391304347826086,
  0.17391304347826086,
  0.20833333333333337,
  0.17391304347826086,
  0.17391304347826086,
  0.25,
  0.0,
  0.25,
  0.9393939393939394,
  0.9736842105263158,
  0.17391304347826086,
  0.65625,
  0.0,
  0.17391304347826086,
  0.28,
  0.935483870967742,
  0.96875,
  0.8918918918918919,
  0.8888888888888888,
  0.9736842105263158,
  0.9166666666666666,
  0.9189189189189189,
  0.9189189189189189,
  0.9189189189189189,
  0.17391304347826086,
  0.6428571428571428,
  0.9736842105263158,
  0.5185185185185186,
  0.65625,
  0.42307692307692313,
  0.42307692307692313,
  0.9473684210526316,
  0.9736842105263158,
  0.9736842105263158,
  0.935483870967742,
  0.8888888888888888,
  0.935483870967742,
  0.935483870967742,
  0.9666666666666667,
  0.8888888888888888,
  0.9285714285714286,
  0.8888888888888888,
  0.935483870967742,
  0.9375,
  0.9666666666666667,
  0.935483870967742,
  0.8888888888888888,
  0.9411764705882353,
  0.8918918918918919,
  0.935483870967742,
  0.888

In [84]:
# listClusterID = []
# filename = "input-files//initialCentroids.txt" 
# f = open(filename)
# data = f.readlines()
# print(data)

# for n in data:
#     mydata = line.rstrip(",\\n")
    
# import csv

# reader = csv.reader(data)
# for row in reader:
#     print(row)

In [85]:
listClusterID = []
filename = "input-files//initialCentroids.txt" 
f = open(filename)
lines = f.read().splitlines()

In [86]:
for i in range(len(lines)):
    lines[i] = lines[i].rstrip(",")
#     lines[i] = int(lines[i])
    listClusterID.append(lines[i])

In [87]:
listClusterID

['323906397735641088',
 '323906483584655360',
 '323906657333682176',
 '323907258301939713',
 '323909308188344320',
 '323913403460636673',
 '324067437886713856',
 '324117950774775809',
 '324138055772561408',
 '324219503401644033',
 '324320247018573824',
 '324346553835868161',
 '324372750330363904',
 '324408472441585664',
 '324422817565257728',
 '324448013999304704',
 '324785120085176320',
 '325059351209443329',
 '325060324992643072',
 '325162944931438592',
 '325253327048822784',
 '325337623910559745',
 '325409910642835456',
 '325701934273134594',
 '325946633986641920']

In [88]:
def getIndexFromId(tweet_id):
    return tweetIDs.index(tweet_id)

In [89]:
assignment = [ 'void' for i in range(len(tweetIDs))]

In [90]:
# until convergence
# assignment
# update centroids
count = 0
while(True):
    count = count+1
    for i in range(len(tweetIDs)):
        minDist = 2
        for m in range(len(listClusterID)):
            distance = arrJD[getIndexFromId(listClusterID[m])][getIndexFromId(tweetIDs[i])]
            if(minDist > distance):
                minDist = distance
                assignment[i] = listClusterID[m]
    # for all cluster id's

    newClusterIDs = []

    for m in range(len(listClusterID)):
        thisClusterID = listClusterID[m]
        listClusterObjects = []

        for i in range(len(tweetIDs)):
            if(assignment[i] == thisClusterID):
                listClusterObjects.append(tweetIDs[i])

        listSumDist = []

        for i in range(len(listClusterObjects)):
            sumDist = 0
            thisTweetID = listClusterObjects[i]
            for j in range(len(listClusterObjects)):
                sumDist = sumDist + arrJD[getIndexFromId(listClusterObjects[i])][getIndexFromId(listClusterObjects[j])]
                listSumDist.append(sumDist)

        newClusterIDs.append(listClusterObjects[listSumDist.index(min(listSumDist))])
        
    if(newClusterIDs == listClusterID):
        break
    listClusterID = newClusterIDs
    
# print(count)

In [91]:
def computeSSE(assignment, listClusterID):
    sse = 0
    for c_id in range(len(listClusterID)):
        for point in range(len(assignment)):
            if(assignment[point] == listClusterID[c_id]):
                sse = sse + arrJD[getIndexFromId(listClusterID[c_id])][getIndexFromId(tweetIDs[point])]*arrJD[getIndexFromId(listClusterID[c_id])][getIndexFromId(tweetIDs[point])]
    return sse

In [92]:
text_file = open("tweets-k-means-output.txt", "w")
for m in range(len(newClusterIDs)):
    string = str(m+1)+'\t\t'
    for i in range(len(tweetIDs)):
        if(assignment[i] == newClusterIDs[m]):
            string = string + tweetIDs[i]+','
    string = string + '\n\n'
    text_file.write(string)

sse = str(computeSSE(assignment, listClusterID))
text_file.write(sse)
text_file.close()